# CITRA DATA

### Path Data

In [2]:
import os
import cv2
import numpy as np
import pandas as pd
import random
from skimage.feature import local_binary_pattern

# Path dataset
dataset_path = "data_mentah/"

# FUNGSI PROGRAM

### Fungsi Klasifikasi Kelompok Umur , Ekstraksi Fitur, dan Augmentasi Gambar

In [3]:
# Fungsi untuk mengonversi umur menjadi kategori umur
def categorize_age(age):
       '''
       Fungsi ini mengonversi umur menjadi kategori umur berdasarkan rentang usia.
       Kategori usia yang dihasilkan:
              0 - Anak-anak (1-10 tahun)
              1 - Remaja (11-18 tahun)
              2 - Dewasa (19-59 tahun)
              -1 - Tidak valid (jika usia tidak dalam rentang yang ditentukan)
       '''
       if 1 <= age <= 10:
              return 0  # Anak-anak
       elif 11 <= age <= 18:
              return 1  # Remaja
       elif 19 <= age <= 59:
              return 2  # Dewasa
       return -1  # Label tidak valid

# Fungsi untuk ekstraksi fitur LBP
def extract_lbp_features(image, radius=2, n_points=16) :
       
       '''
       Fungsi ini menghitung fitur LBP dari gambar untuk digunakan dalam pengenalan wajah.
       LBP digunakan untuk mengekstraksi informasi tekstur dari gambar wajah.
       
       Parameter:
       image - Gambar input dalam format grayscale.
       radius - Jarak dari titik pusat pada LBP (default: 2).
       n_points - Jumlah titik tetangga pada LBP (default: 16).
       
       Menghasilkan:
       lbp_hist - Histogram yang dinormalisasi dari pola LBP.
       '''
       
       # Hitung LBP pada gambar
       lbp = local_binary_pattern(image, n_points, radius, method="uniform")
       
       # Hitung histogram dari LBP
       lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
       
       # Normalisasi histogram
       lbp_hist = lbp_hist.astype("float")
       lbp_hist /= (lbp_hist.sum() + 1e-6)  # Normalisasi agar jumlah histogram = 1
       return lbp_hist

# Fungsi untuk enhancement citra
def enhance_image(image):
       
       '''
       Fungsi ini digunakan untuk meningkatkan kualitas gambar dengan menerapkan filter penajaman (sharpening).
       Jika gambar berwarna (BGR), akan dikonversi ke grayscale terlebih dahulu.
       
       Parameter:
       image - Gambar input yang bisa berupa gambar berwarna atau grayscale.
       
       Menghasilkan:
       enhanced_img - Gambar yang telah ditingkatkan kualitasnya dengan filter sharpening.
       '''
       
       # Periksa apakah citra sudah grayscale
       if len(image.shape) == 3:  # Jika memiliki 3 kanal (BGR)
              gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
       else:
              gray = image  # Jika sudah grayscale, gunakan langsung
       
       # Mendefinisikan kernel sharpening (3x3)
       kernel = np.array([
              [0, -1, 0],
              [-1, 5, -1],
              [0, -1, 0]
       ])
       
       # Melakukan operasi filter untuk sharpening
       enhanced_img = cv2.filter2D(gray, -1, kernel)
       
       return enhanced_img

# DATA PROCESSING & EKSTRAKSI FITUR

### Read Gambar dan Menyimpannya di List

In [4]:
# Inisialisasi list untuk menyimpan data
data = []

# Mendapatkan daftar file gambar dalam folder dataset
image_files = [f for f in os.listdir(dataset_path) if f.endswith((".jpg", ".png"))]
image_data_by_age = {}

# Mengelompokkan gambar berdasarkan kategori umur
for image_file in image_files:
    parts = image_file.split('_')
    age = int(parts[0])  # Mengambil usia dari nama file
    age_category = categorize_age(age)
    
    if age_category != -1:
        if age_category not in image_data_by_age:
            image_data_by_age[age_category] = []
        image_data_by_age[age_category].append(image_file)

# Proses gambar yang ada untuk ekstraksi fitur
for age_category, files in image_data_by_age.items():
    for image_file in files:
        image_path = os.path.join(dataset_path, image_file)
        
        # Baca gambar dan konversi ke grayscale
        image = cv2.imread(image_path)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Ubah ke grayscale untuk LBP
        
        # Lakukan enhancement pada gambar
        enhanced_img = enhance_image(gray_image)
        
        # Deteksi wajah
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        faces = face_cascade.detectMultiScale(enhanced_img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # Jika wajah terdeteksi, crop wajah dan simpan
        if len(faces) > 0:
            for (x, y, w, h) in faces:
                # Crop wajah
                face_image = enhanced_img[y:y+h, x:x+w]
                face_image = cv2.resize(face_image, (256, 256))  # Resize wajah agar konsisten
                
                # Ekstraksi fitur LBP
                lbp_features = extract_lbp_features(face_image)
                
                # Menggabungkan label umur dan fitur LBP ke dalam list data
                data.append([age_category] + lbp_features.tolist())

### Menyimpan Data Gambar ke File CSV

In [5]:
# Konversi ke DataFrame Pandas dan simpan ke CSV
df = pd.DataFrame(data, columns=['age_category'] + [f'pixel_{i}' for i in range(len(lbp_features))])
df.to_csv("df_sampled.csv", index=False)

# CEK INFORMASI DATA

### Read Data CSV

In [6]:
# Membaca data dari CSV
df = pd.read_csv("df_sampled.csv")
print(df.head())

   age_category   pixel_0   pixel_1   pixel_2   pixel_3   pixel_4   pixel_5  \
0             1  0.079910  0.052582  0.028976  0.018967  0.015472  0.017868   
1             1  0.026154  0.019470  0.023331  0.026443  0.028564  0.038712   
2             1  0.052902  0.035355  0.023117  0.016815  0.017944  0.025467   
3             1  0.075363  0.044846  0.033310  0.022293  0.018692  0.020416   
4             1  0.030167  0.022156  0.021393  0.023773  0.025482  0.036789   

    pixel_6   pixel_7   pixel_8   pixel_9  pixel_10  pixel_11  pixel_12  \
0  0.022842  0.034851  0.048630  0.042404  0.025024  0.021805  0.019043   
1  0.056015  0.109863  0.192642  0.130234  0.059433  0.039124  0.031494   
2  0.035995  0.057861  0.072067  0.064041  0.035065  0.029861  0.021515   
3  0.028229  0.041565  0.054016  0.044220  0.028824  0.021088  0.018524   
4  0.056122  0.104660  0.147079  0.119156  0.060287  0.040314  0.028366   

   pixel_13  pixel_14  pixel_15  pixel_16  pixel_17  
0  0.021622  0.03111

### Memeriksa Konsistensi Kategori Usia

In [7]:
# Memeriksa distribusi kategori usia
print(df['age_category'].value_counts())

age_category
0    127
2    118
1    116
Name: count, dtype: int64


# DATA SPLITTING

In [36]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# Memisahkan fitur dan target
X = df.drop(columns=['age_category'])
y = df['age_category']

# Menggunakan SMOTE untuk menyeimbangkan kategori
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Split data yang telah diseimbangkan
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.1, random_state=42)


# PELATIHAN MODEL (RANDOM FOREST)

In [34]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Inisialisasi model Random Forest dengan parameter awal
model = RandomForestClassifier(n_estimators=200, random_state=42, class_weight='balanced')

# Melatih model
model.fit(X_train, y_train)

# Evaluasi model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Akurasi Model Random Forest:", accuracy * 100, "%")
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Akurasi Model Random Forest: 75.0 %

Classification Report:
               precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       1.00      0.50      0.67         2
           2       1.00      1.00      1.00         1

    accuracy                           0.75         4
   macro avg       0.83      0.83      0.78         4
weighted avg       0.88      0.75      0.75         4



In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Anak-anak', 'Remaja', 'Dewasa'], yticklabels=['Anak-anak', 'Remaja', 'Dewasa'])
plt.xlabel('Prediksi')
plt.ylabel('Sebenarnya')
plt.show()

In [ ]:
import joblib

# Menyimpan model
joblib.dump(model, "famscreen_model.pkl")

In [ ]:
# Memuat model Random Forest yang telah disimpan
model = joblib.load('famscreen_model.pkl')

# PENGUJIAN MENGGUNAKAN INPUTAN GAMBAR DARI APLIKASI

In [69]:
import matplotlib.pyplot as plt
import cv2
import numpy as np

# Fungsi untuk mengubah angka prediksi menjadi kategori usia (label teks)
def age_category_to_label(age_category):
    if age_category == 0:
        return "Anak-anak"
    elif age_category == 1:
        return "Remaja"
    elif age_category == 2:
        return "Dewasa"
    return "Tidak Valid"

# Fungsi untuk mendeteksi wajah dan ekstraksi fitur LBP dari gambar baru
def detect_and_predict(image_path, model, face_cascade, radius=2, n_points=16):
    # Membaca gambar dan konversi ke grayscale
    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Lakukan enhancement pada gambar
    enhanced_img = enhance_image(gray_image)

    # Deteksi wajah dalam gambar
    faces = face_cascade.detectMultiScale(enhanced_img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Jika wajah terdeteksi
    if len(faces) > 0:
        for (x, y, w, h) in faces:
            # Crop wajah
            face_image = enhanced_img[y:y+h, x:x+w]
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            face_image_resized = cv2.resize(face_image, (256, 256))

            # Ekstraksi fitur LBP dari wajah yang sudah diaugmentasi
            lbp_features = extract_lbp_features(face_image_resized, radius=radius, n_points=n_points)

            # Prediksi menggunakan model Random Forest
            prediction = model.predict([lbp_features])  # Model menerima array 2D, jadi fitur LBP harus dalam list
            predicted_age_category = prediction[0]
            
            # Mengonversi angka prediksi menjadi kategori usia (label teks)
            predicted_age_label = age_category_to_label(predicted_age_category)

            # Visualisasi hasil
            plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            plt.title(f'Prediksi Umur: {predicted_age_label}')
            plt.axis('off')
            plt.show()

            return predicted_age_category, predicted_age_label
    else:
        print("Tidak ada wajah yang terdeteksi dalam gambar.")
        return None, None

# Contoh penggunaan deteksi dengan inputan gambar baru
image_path = "Images/imageFile.jpg"  
detected_age_category, predicted_age_label = detect_and_predict(image_path, model, face_cascade)

if detected_age_category is not None:
    print(f"Kategori Umur Prediksi: {predicted_age_label}")

Tidak ada wajah yang terdeteksi dalam gambar.
